# Import

In [1]:
!pip install mysql-connector-python

In [35]:
import mysql.connector
from datetime import date

# Connection

In [3]:
cnx = mysql.connector.connect(host='172.26.0.2',port='3306',database='nico_studienarbeit',user='studienarbeit',password='dbstuar2020')

In [4]:
cursor = cnx.cursor()

# Querys

If string include ' ' to resole errors in the statement!

## Insert routines
1. Buch: Sorte, Verlag, Buch
2. Hörbuch: Sorte, Verlag, Buch, Person, Sprecher, Hörbuch
3. Ebook: Sorte, Verlag, Buch, Person, Ebook
4. Person: Person
5. Ausleiher: Person, Ausleiher
6. Sorte: Sorte
7. Autor: Person, Autor
8. Schlagwort: Schlagwort
9. Maler: Person, Maler

### Buch
INSERT_SORTE, INSERT_VERLAG, INSERT_AUTOR

In [5]:
INSERT_SORTE = "INSERT INTO Sorte(Name,Beschreibung) VALUES(%s,'%s)"

In [6]:
INSERT_VERLAG = "INSERT INTO Verlag(Kurzname, Name, Postleitzahl, Straße, Internetaddresse, Beschreibung) VALUES(%s,%s,%s,%s,%s,%s)"

In [7]:
INSERT_BUCH = "INSERT INTO Buch(ISBN,Titel,Untertitel,VerlagId,Erscheinungsjahr,SorteId,Kurzbeschreibung,Preis,Auflage,Sprache,Titelbild) VALUES(%s,%s,%s,%d,%d,%d,%d,%s,%d,%s,%d);"

In [8]:
INSERT_AUTORBUCHZUORD = "INSERT INTO AutorBuchZuord(AutorId,ISBN) VALUES(%d,%s);"

In [9]:
def insert_book(val_schlagwort, val_verlag, val_buch):
    # Name,Beschreibung
    cursor.execute(INSERT_SCHLAGWORT, val_schlagwort)
    # Kurzname, Name, Postleitzahl, Straße, Internetaddresse, Beschreibung
    cursor.execute(INSERT_VERLAG, val_verlag)
    # ISBN,Titel,Untertitel,VerlagId,Erscheinungsjahr,SorteId,Kurzbeschreibung,Preis,Auflage,Sprache,Titelbild
    cursor.execute(INSERT_BUCH, val_buch)
    
    cursor.execute(INSERT_PERSON, val_person)
    
    cursor.execute(INSERT_AUTOR, val_autor)
    cnx.commit()
    return

### Hörbuch
INSERT_SORTE, INSERT_VERLAG, INSERT_BUCH, INSERT_SPRECHER

In [10]:
INSERT_PERSON = "INSERT INTO Person(Vorname,Name,Email,Geburtsdatum) VALUES(%s,%s,%s,%s)"

In [11]:
INSERT_SPRECHER = "INSERT INTO Sprecher(PersonId,Beschreibung) VALUES(%s,%s)"

In [12]:
INSERT_HOERBUCH = "INSERT INTO Hoerbuch(ISBN,BuchISBN,SprecherId,VerlagId) VALUES(%s,%s,%s,%s)"

In [13]:
def insert_hoerbuch():
    pass

### Ebook
INSERT_SORTE, INSERT_VERLAG, INSERT_BUCH, INSERT_PERSON

In [14]:
INSERT_EBOOK = "INSERT INTO Ebooks(ISBN,BuchISBN,Dateiformat) VALUES(%s,%s,%s)"

### Ausleiher
INSERT_PERSON

In [15]:
INSERT_AUSLEIHER = "INSERT INTO Ausleiher(PersonenId,Strasse,Postleitzahl,Ort,Telefonnummer) VALUES(%s,%s,%s,%s,%s)"

### Autor
INSERT_PERSON

In [16]:
INSERT_AUTOR = "INSERT INTO AUTOR(PersonenId,Beschreibung) VALUES(%s,%s)"

### Schlagwort

In [17]:
INSERT_SCHLAGWORT = "INSERT INTO Schlagwort(Wort,Beschreibung) VALUES(%s,%s)"

### Maler
INSERT_PERSON

In [18]:
INSERT_MALER = "INSERT INTO Maler(PersonenId,Beschreibung) VALUES(%s,%s)"

### AutorBuchZuord
INSERT_PERSON, INSERT_AUTOR, INSERT_SORTE, INSERT_VERLAG, INSERT_BUCH

In [19]:
INSERT_AUTORBUCHZUORD = "INSERT INTO AutorBuchZuord(AutorId, ISBN) VALUES(%s,%s)"

### NichtTextMedien
INSERT_SORTE

In [20]:
INSERT_NICHTTEXTMEDIEN = "INSERT INTO NichtTextMedien(Titel,Untertitel,Erscheinungsjahr,Kurzbeschreibung,SorteId,Typ) VALUES(%s,%s,%s,%s,%s,%s)"

### Bild
INSERT_SORTE, INSERT_NICHTTEXTMEDIEN, INSERT_MALER

In [21]:
INSERT_BILD = "INSERT INTO Bild(NichtTextMedienId,Bild,MalerId) VALUES(%s,%s,%s)"

### Video
INSERT_SORTE, INSERT_NICHTTEXTMEDIEN

In [22]:
INSERT_VIDEO = "INSERT INTO Video(NichtTextMedienId,Sprache) VALUES(%s,%s)"

### Ausleihe
INSERT_AUSLEIHER, INSERT_NICHTTEXTMEDIEN/INSERT_BUCH/INSERT_EBOOK/INSERT_HOERBUCH

In [23]:
INSERT_AUSLEIHE = "INSERT INTO Ausleihe(AusleiherId,MediumId,Ausleihdatum,Rueckgabedatum) VALUES(%s,%s,%s,%s)"

# Testqueries

In [40]:
#Vorname,Name,Email,Geburtsdatum
cursor.execute("INSERT INTO Person(Vorname,Name,Email,Geburtsdatum) VALUES(%s,%s,%s,%s)", ("Ludwig", "Test", "ludwig@test.de", date(2002,4,13)))

In [41]:
cnx.commit()

In [42]:
cursor.lastrowid

5

In [43]:
cursor.execute("SELECT * FROM Person WHERE Vorname=%s AND Name=%s AND Email=%s AND Geburtsdatum=%s;", ("Ludwig", "Test", "ludwig@test.de", date(2002,4,13)))

In [34]:
cursor.statement

"SELECT * FROM Person WHERE Vorname='Peter' AND Name='Test' AND Email='peter@test.de' AND Geburtsdatum='2019-01-01';"

In [44]:
print(cursor.fetchone())

(5, 'Ludwig', 'Test', 'ludwig@test.de', datetime.date(2002, 4, 13))


# Execute

# Close

In [47]:
cursor.close()
cnx.close()